In [ ]:
#|default_exp ww_metashapelib

# ww_metashapelib
> A Python library for reading and analysing MetaShape reference and camera calibration files.

#| hide
## Imports

In [ ]:
#|export
import datetime as              dt
import csv
from   glob    import           glob
import re
import os
from   io    import             StringIO
from pathlib import             Path, PurePosixPath, PureWindowsPath
import xml.etree.ElementTree as ET

In [ ]:
#|export
import pandas   as              pd

In [ ]:
#|export
import numpy    as              np

#| hide
## Module Data

#| hide
### Static Data.

In [ ]:
#|export
# The asof data for this version of the library.
asof = '2024-0407-0052'

#| hide
### Test and Configuration Data
These cells setup any files used for testing the functions and classes in this module.  Configurations for testing on Linux/Colab, Linux/wsl2-Jupyter, Linux/Jupyter, and Windows/Jupyter

#| hide
### All Systems

In [ ]:
#| hide
# Set these for all systems.
if __name__ == "__main__":
  metashape_cal_filename = 'we-1500-A1-agisoft-camcal.xml'
  metashape_ref_filename = 'we-1500-A1-agisoft-camcal-ref.txt'
  inpho_filename         = 'we-1500-A1-inpho-camcal.txt'
  opencv_filename        = 'we-1500-A1-opencv-camcal.xml'
  ref_test_mask          = '*-ref.txt'

#| hide
### Colab 

In [ ]:
#| hide
# Colab setup.
# See: https://openincolab.com/ to generate a Colab badge.
if __name__ == "__main__":
  if 'google.colab' not in str(get_ipython()):
    print('Not running on CoLab')
    IN_COLAB = False
  else:
    print('Running on Linux/CoLab')
    ######################### xximport google.colab
    IN_COLAB = True
    if os.path.exists('/content/drive') == False:
      from google.colab import drive
      drive.mount('/content/drive')
    else:
      print('Google Drive already mounted.')
      test_data_dir          = '/content/drive/MyDrive/Colab Notebooks/WWpylib/test_data/'
      metashape_cal_filename = test_data_dir + metashape_cal_filename
      metashape_ref_filename = test_data_dir + metashape_ref_filename
      inpho_filename         = test_data_dir + inpho_filename
      opencv_filename        = test_data_dir + opencv_filename
      print('\n**** Testing Files ****')
      print(f'{metashape_cal_filename    = }')
      print(f'{metashape_ref_filename    = }')
      print(f'{inpho_filename            = }')
      print(f'{opencv_filename           = }')
      print("\n**** Reference data test files ****")
      ref_test_files = glob(f'{test_data_dir}/{ref_test_mask}')
      for f in ref_test_files:
        print(f)

Not running on CoLab


#| hide
### Windows/Juypter

In [ ]:
#| hide
# Windows setup.
if __name__ == "__main__" and os.name == 'nt':
  print('Running on Windows/Jupyter.')

#| hide
### Linux/Jupyter

In [ ]:
#| hide
# Linux/Jupyter setup.
if __name__ == "__main__" and os.name == 'posix' and not IN_COLAB:
  print('Running on Linux/Jupyter.')
  test_data_dir          = '../test_data/'
  print(f'{test_data_dir = }')
  metashape_cal_filename = test_data_dir + metashape_cal_filename
  metashape_ref_filename = test_data_dir + metashape_ref_filename
  inpho_filename         = test_data_dir + inpho_filename
  opencv_filename        = test_data_dir + opencv_filename
  print('\n**** Testing Files ****')
  print(f'{metashape_cal_filename    = }')
  print(f'{metashape_ref_filename    = }')
  print(f'{inpho_filename            = }')
  print(f'{opencv_filename           = }')
  print("\n**** Reference data test files ****")
  ref_test_files = glob(f'{test_data_dir}/{ref_test_mask}')
  for f in ref_test_files:
    print(f)

Running on Linux/Jupyter.
test_data_dir = '../test_data/'

**** Testing Files ****
metashape_cal_filename    = '../test_data/../test_data/../test_data/we-1500-A1-agisoft-camcal.xml'
metashape_ref_filename    = '../test_data/../test_data/../test_data/we-1500-A1-agisoft-camcal-ref.txt'
inpho_filename            = '../test_data/../test_data/../test_data/we-1500-A1-inpho-camcal.txt'
opencv_filename           = '../test_data/../test_data/../test_data/we-1500-A1-opencv-camcal.xml'

**** Reference data test files ****
../test_data/ns1500-ref.txt
../test_data/ew1500-ref.txt
../test_data/ew2000-ref.txt
../test_data/ew1200-ref.txt


#| hide
## Functions

### class MetaShapeReference

Class instance attributes described in the table below are filled in by the `read_metashape_reference_file_into_dataframe()`
function.

*Table generated with: [Table generator](https://www.tablesgenerator.com/markdown_tables)*

| Attribute   | Type      | Description                                                                           |
|-------------|-----------|---------------------------------------------------------------------------------------|
| filename    | str       | String containing the filename.                                                       |
| df          | DataFrame | A pandas Dataframe containing the values extracted from the file.                     |
| meta        | DataFrame | The contents of the metadata extracted from the header of the file.                   |
| total_error | class     | A class object containing the elements found in the `Total Error` line from the file. |
| data        | str       | A copy of the entire string data read from the file.                                  |     |he file.

In [ ]:
#|export
class MetaShapeReference:
  """
  This class holds data extracted from a MetaShape reference file.
  """
  def __init__(self,
               filename:str  # String containing the filename.
              ):
    self.filename:str      = filename  # String containing the filename.
    self.df:obj            = None      # Pandas DataFrame with the reference data.
    self.meta:str          = None      # The contents of the header line describing datums, etc.
    self.total_error:obj   = None      # A class popuated with the data extracted from the `Total Error` line.
    self.data:list         = None      # A list of all lines (strings) extracted from the file.

#| hide
### def read_metashape_reference_file_into_dataframe()

In [ ]:
#|export
def read_metashape_reference_file_into_dataframe(
    filename           # Metashape reference filename to read.
    ) -> pd.DataFrame: # MetashapeReference class data structure.
  """
  Read a Metashape reference file into a MetashapeReference class
  and save the data in a Pandas dataframe.  Also save the filename, the
  metadata from the comment above the header line, the total_error as
  a Pandas Dataframe, and the contents of the specified file in string,
  in the returned data structure.
  """

  # Create the Matashape data structure, and save the filename in it.
  ref = MetaShapeReference( filename )

  # Extract the root filename
  ref.root_filename = PurePosixPath(filename).name

  # Read the file into a list of lines. so we can get rid of the "#" at the start
  # of the header line.
  with open(filename, 'r') as f:
    data = f.readlines()

  # Save the whole thing for possible debugging.
  ref.data = data

  # Save the meta data in the data structure in case the user wants it.
  ref.meta = data[0]

  # Get rid of the "#" at the start of the header line.
  if data[1][0:6] == '#Label':
    data[1] = data[1][1:]

  # Convert the list to a string
  ds = ''.join( data )

  # Read the string into a Pandas dataframe and store in the data structure.
  ref.df = pd.read_csv( StringIO(ds), comment='#')

  # Read the last line of the file into a Pandas dataframe and store in the data structure.
  # This line has the total error for several columns.
  ref.total_error = pd.read_csv( StringIO( data[-1] ), names=ref.df.columns ).dropna(axis=1)
  ref.total_error.replace( to_replace="#Total error", value=ref.root_filename, inplace=True )

  # Return the data structure.
  return ref

Generate a list of test reference files.

In [ ]:
#| hide
# Gather test reference files.
if __name__ == '__main__':
  files = glob(f'{test_data_dir}/*-ref.txt')
  display(files)

['../test_data/ns1500-ref.txt',
 '../test_data/ew1500-ref.txt',
 '../test_data/ew2000-ref.txt',
 '../test_data/ew1200-ref.txt']

Test the function `read_metashape_reference_file_into_dataframe()` by reading in the 
first file from the list of test files.

In [ ]:
#| hide
# Test with a single ref file.
if __name__ == "__main__":
  ref = read_metashape_reference_file_into_dataframe( files[0] )
  display(ref.df.head(3), ref.df.tail(3), ref.total_error)

,Label,X/Longitude,Y/Latitude,Z/Altitude,Yaw,Pitch,Roll,Accuracy_X/Y/Z_(m),Error_(m),X_error,...,Error_(deg),Yaw_error,Pitch_error,Roll_error,X_est,Y_est,Z_est,Yaw_est,Pitch_est,Roll_est
0,20190819-172136.9216.jpg,-95.460133,29.121022,456.9397,181.264,3.234,-0.131,0.1,0.009178,-0.003472,...,0.703587,-0.469101,-0.522711,-0.041860,-95.460133,29.121022,456.940095,180.794899,2.711289,-0.172860
1,20190819-172137.4215.jpg,-95.460132,29.120692,456.2952,181.299,3.313,-0.406,0.1,0.060060,0.027659,...,0.690168,-0.446910,-0.523026,0.055196,-95.460132,29.120692,456.274827,180.852090,2.789974,-0.350804
2,20190819-172137.9216.jpg,-95.460130,29.120361,455.5638,181.314,3.406,-0.745,0.1,0.053294,-0.015039,...,0.696537,-0.458681,-0.523640,0.024024,-95.460130,29.120362,455.545118,180.855319,2.882360,-0.720976


,Label,X/Longitude,Y/Latitude,Z/Altitude,Yaw,Pitch,Roll,Accuracy_X/Y/Z_(m),Error_(m),X_error,...,Error_(deg),Yaw_error,Pitch_error,Roll_error,X_est,Y_est,Z_est,Yaw_est,Pitch_est,Roll_est
62,20190819-172208.9216.jpg,-95.460071,29.099850,450.4040,182.630,4.662,0.758,0.1,0.052272,-0.006767,...,0.682765,-0.482940,-0.482448,-0.013455,-95.460071,29.099850,450.430421,182.147060,4.179552,0.744545
63,20190819-172209.4216.jpg,-95.460073,29.099520,450.7224,182.571,4.543,0.354,0.1,0.058395,0.009629,...,0.681117,-0.434328,-0.518919,0.077483,-95.460073,29.099519,450.746367,182.136672,4.024081,0.431483
64,20190819-172209.9216.jpg,-95.460074,29.099189,450.9248,182.517,4.464,0.040,0.1,0.036471,-0.030250,...,0.755245,-0.468619,-0.547082,-0.226917,-95.460074,29.099189,450.944385,182.048381,3.916918,-0.186917


,Label,Error_(m),X_error,Y_error,Z_error,Error_(deg),Yaw_error,Pitch_error,Roll_error
0,ns1500-ref.txt,0.042235,0.027437,0.026548,0.018061,0.700134,0.459896,0.520026,0.090862


#| hide
### def read_metashape_reference_file_total_errors()

In [ ]:
#|export
def read_metashape_reference_file_total_errors( 
    filename:str   # Filename of a MetaShape reference file. 
) ->object:        # Pandas dataframe containing the entries from the `Total Error` line  in the file.
  """
  Reads a `MetaShape` reference file into a Pandas Dataframe.
  """
  df = read_metashape_reference_file_into_dataframe( filename )
  s = df.df['Label'][0].split('.')[0]
  dtime = dt.datetime.strptime(s, '%Y%m%d-%H%M%S')
  df.total_error['datetime'] = dtime
  return df.total_error

Test the file and display the resulting Pandas Dataframe.

In [ ]:
#| hide
# Test read_metashape_reference_file_total_errors()
if __name__ == '__main__':
  df = read_metashape_reference_file_total_errors( ref_test_files[0] )
  display(df)

,Label,Error_(m),X_error,Y_error,Z_error,Error_(deg),Yaw_error,Pitch_error,Roll_error,datetime
0,ns1500-ref.txt,0.042235,0.027437,0.026548,0.018061,0.700134,0.459896,0.520026,0.090862,2019-08-19 17:21:36


Test read_metashape_reference_file_total_errors() with multiple files

In [ ]:
#| hide
# Test read_metashape_reference_file_total_errors() with multiple files
if __name__ == '__main__':
  df = pd.DataFrame() # read_metashape_reference_file_total_errors( ref_test_files[0] )
  for f in ref_test_files:
    dfn = read_metashape_reference_file_total_errors( f )
    df = pd.concat( [df, dfn] )
  display(df)

,Label,Error_(m),X_error,Y_error,Z_error,Error_(deg),Yaw_error,Pitch_error,Roll_error,datetime
0,ns1500-ref.txt,0.042235,0.027437,0.026548,0.018061,0.700134,0.459896,0.520026,0.090862,2019-08-19 17:21:36
0,ew1500-ref.txt,0.053197,0.040021,0.030392,0.017453,0.666923,0.444327,0.478524,0.135554,2019-08-19 17:32:33
0,ew2000-ref.txt,0.054781,0.033468,0.039589,0.017709,0.647979,0.459748,0.436593,0.133774,2019-08-19 17:08:47
0,ew1200-ref.txt,0.230996,0.041630,0.128267,0.187547,0.639768,0.439518,0.450201,0.115958,2019-08-19 16:56:31


In [ ]:
#|export
class MetaShape_ref_total_errors:
  """
  This class is populated with the `Total Error` values from the Agisoft MetaShape reference export file.  It 
  is populated by the `read_metashape_reference_dir_total_errors()` function. Once populated it will 
  containt the path name `self.path` of the directory where the reference files are located, and a Pandas
  Dataframe `self.df` which contains the actual data from the `Total Errors`.
  """
  def __init__(self):
    self.path   = None
    self.df     = None

#| hide
### def read_metashape_reference_dir_total_errors()

In [ ]:
#|export
def read_metashape_reference_dir_total_errors(
    dir_path:str,         # Path to MetaShape reference data files.
    mask:str='*-ref.txt'  # Mask to select the reference files.
    ) -> object:          # Class with path and dataframe attributes
  """
  Reads all MetaShape files matching `mask` in the specified `path`.  The `Total Error`
  from each file is extracted and loaded into a row in a Pandas Dataframe which is 
  returned to the user for analysis.  
  """
  df = pd.DataFrame()
  rv = MetaShape_ref_total_errors()
  rv.path = dir_path
  for f in glob(f'{dir_path}/{mask}'):
    dfn = read_metashape_reference_file_total_errors( f )
    df = pd.concat( [df, dfn] )
  v = df.pop('datetime')            # Move 'datetime' to be the second column
  df.insert(1, 'datetime', v)
  df.reset_index(drop=True, inplace=True)
  rv.df = df
  return rv

#| hide
#### Test def read_metashape_reference_dir_total_errors()

Test read_metashape_reference_file_total_errors() with multiple files

In [ ]:
#| hide
# Test read_metashape_reference_file_total_errors() with multiple files
if __name__ == '__main__':
  rv = read_metashape_reference_dir_total_errors( test_data_dir )
  display(rv.path, rv.df)

'../test_data/'

,Label,datetime,Error_(m),X_error,Y_error,Z_error,Error_(deg),Yaw_error,Pitch_error,Roll_error
0,ns1500-ref.txt,2019-08-19 17:21:36,0.042235,0.027437,0.026548,0.018061,0.700134,0.459896,0.520026,0.090862
1,ew1500-ref.txt,2019-08-19 17:32:33,0.053197,0.040021,0.030392,0.017453,0.666923,0.444327,0.478524,0.135554
2,ew2000-ref.txt,2019-08-19 17:08:47,0.054781,0.033468,0.039589,0.017709,0.647979,0.459748,0.436593,0.133774
3,ew1200-ref.txt,2019-08-19 16:56:31,0.230996,0.041630,0.128267,0.187547,0.639768,0.439518,0.450201,0.115958


#| hide
### class MetaShape_Cal_Data()

Class holds camera calibration data extracted from a `MetaShape` camera calibration file.  This 
class is populated by the `read_metashape_camera_cal_file()` function.

In [ ]:
#|export
class MetaShape_Cal_Data():
  def __init__(self):
    self.filename   = None
    self.datetime   = None
    self.projection = None
    self.units      = None
    self.pixel_size = None
    self.f          = None
    self.cx         = None
    self.cy         = None
    self.width      = None
    self.height     = None
    self.error      = None

#| hide
### def read_metashape_camera_cal_file()

In [ ]:
#|export
def read_metashape_camera_cal_file(
    filename,              # The MetaShape xml calibration file to read.
    pixel_size  = 5.5e-6,  # Pixel size in meters.
    units = 'pixels',      # Can be "pixels", "m", or "mm"
    rv = 'dict',           # R value. Can be "dict" or "df" or "class"
    debug=False            #
    ) -> dict:             #
  """
  Read a Metashape xml camera calibration file into a dictionary. Values
  can be converted from pixels to meters, or millimeters. Be sure to
  specify the pixel size in meters.
  """
  cd = MetaShape_Cal_Data()    # class to hold data.
  cal = { "filename" : Path(filename),   # filename.split("/")[-1],
         "pixel_size": pixel_size,
          "units"     : units
        }
  cd.filename   = Path(filename)
  cd.pixel_size = pixel_size
  cd.units      = units
  try:
    tree = ET.parse( filename )
    root = tree.getroot()
  except:
    cd.error = f"Et.parse( {filename} ) failed."
    return cd

  if debug:
    print(f'debug: root.tag:{root.tag}')
  if root.tag == 'calibration':
    for child in root:
      if debug:
        print(f'debug: {child.tag:10s} = {child.text}')
      if child.tag == 'projection':
        cd.projection = cal[child.tag] = child.text
      elif child.tag == 'date':
        cd.datetime = cal[child.tag] = dt.datetime.strptime( child.text,"%Y-%m-%dT%H:%M:%SZ" )
      else:
        cal[child.tag] = float(child.text)

    if units != 'pixels':
      if units   == 'm':
        scale = 1.0
      elif units == 'mm':
        scale = 1000.0

      cd.f      = cal['f' ]      * cd.pixel_size * scale
      cd.cx     = cal['cx']     * cd.pixel_size * scale
      cd.cy     = cal['cy']     * cd.pixel_size * scale
      cd.width  = cal['width']  * cd.pixel_size * scale
      cd.height = cal['height'] * cd.pixel_size * scale

      for k in ['f', 'cx', 'cy', 'width', 'height']:
        cal[k] = cal[k] * cal['pixel_size'] * scale

    if rv == 'class':
      cd.error = 'None'
      return cd
    elif rv == 'dict':
      return cal
    elif rv == 'df':
      df = pd.DataFrame( [ cal ] )
      return df
  else:
    cd.error = f"Not a MetaShape calibration file, lacks <calibration> tag."
    return cd

#| hide
#### Test def read_metashape_camera_cal_file() with rv='class'

In [ ]:
if __name__ == "__main__":
  cal = read_metashape_camera_cal_file( metashape_cal_filename, units='m', rv='class' )

In [ ]:
if __name__ == "__main__":
  print(
      f"{cal.error      = }\n"
      f"{cal.datetime   = }\n"
      f"{cal.filename   = }\n"
      f"{cal.pixel_size = }\n"
      f"{cal.width      = }\n"
      f"{cal.height     = }\n"
      f"{cal.projection = }\n"
      f"{cal.units      = }\n"
      f"{cal.f          = }\n"
      f"{cal.cx         = }\n"
      f"{cal.cy         = }\n"
      )

  # Dump contents via keys and values.
  for k in cal.__dict__.keys():
    print(f'{k:15s}= {cal.__dict__[k]}')

  # Dump all the parts of the filename
  display(cal.filename.parts)

  # Show the filename without the path
  print(f"Filename without the path: {cal.filename.name}")
  print(f"Pathname without the file: {cal.filename.parent}")

cal.error      = 'None'
cal.datetime   = datetime.datetime(2024, 4, 4, 4, 49, 48)
cal.filename   = PosixPath('../test_data/we-1500-A1-agisoft-camcal.xml')
cal.pixel_size = 5.5e-06
cal.width      = 0.026928
cal.height     = 0.017952
cal.projection = 'frame'
cal.units      = 'm'
cal.f          = 0.028751982753143952
cal.cx         = 6.456287348269344e-05
cal.cy         = -4.784378165888971e-05

filename       = ../test_data/we-1500-A1-agisoft-camcal.xml
datetime       = 2024-04-04 04:49:48
projection     = frame
units          = m
pixel_size     = 5.5e-06
f              = 0.028751982753143952
cx             = 6.456287348269344e-05
cy             = -4.784378165888971e-05
width          = 0.026928
height         = 0.017952
error          = None


('..', 'test_data', 'we-1500-A1-agisoft-camcal.xml')

Filename without the path: we-1500-A1-agisoft-camcal.xml
Pathname without the file: ../test_data


In [ ]:
if __name__ == "__main__":
  cal = read_metashape_camera_cal_file( metashape_cal_filename, units='m', rv='df' )
  display(cal)

,filename,pixel_size,units,projection,width,height,f,cx,cy,k1,k2,k3,p1,p2,date
0,../test_data/we-1500-A1-agisoft-camcal.xml,0.000005,m,frame,0.026928,0.017952,0.028752,0.000065,-0.000048,-0.094197,0.115036,-0.032238,-0.000257,-0.000354,2024-04-04 04:49:48


In [ ]:
if __name__ == "__main__":
  cal = read_metashape_camera_cal_file( metashape_cal_filename, debug=True )
  display(cal)

debug: root.tag:calibration
debug: projection = frame
debug: width      = 4896
debug: height     = 3264
debug: f          = 5227.6332278443551
debug: cx         = 11.738704269580627
debug: cy         = -8.6988693925254026
debug: k1         = -0.094196634563145282
debug: k2         = 0.11503642426152168
debug: k3         = -0.032238313340550974
debug: p1         = -0.00025662254090614374
debug: p2         = -0.00035361346010062737
debug: date       = 2024-04-04T04:49:48Z


{'filename': PosixPath('../test_data/we-1500-A1-agisoft-camcal.xml'),
 'pixel_size': 5.5e-06,
 'units': 'pixels',
 'projection': 'frame',
 'width': 4896.0,
 'height': 3264.0,
 'f': 5227.633227844355,
 'cx': 11.738704269580627,
 'cy': -8.698869392525403,
 'k1': -0.09419663456314528,
 'k2': 0.11503642426152168,
 'k3': -0.032238313340550974,
 'p1': -0.00025662254090614374,
 'p2': -0.0003536134601006274,
 'date': datetime.datetime(2024, 4, 4, 4, 49, 48)}

In [ ]:
# Test with the wrong file, a non xml file.
if __name__ == "__main__":
  cal = read_metashape_camera_cal_file( inpho_filename, debug=True )
  display(cal.error)

'Et.parse( ../test_data/we-1500-A1-inpho-camcal.txt ) failed.'

In [ ]:
# Test with an xml file which is not a Metashape cal file.
if __name__ == "__main__":
  cal = read_metashape_camera_cal_file( opencv_filename, units='mm' )
  display(cal.error)

'Not a MetaShape calibration file, lacks <calibration> tag.'

#| hide
#### Test read_metashape_camera_cal_file() with rv='df

In [ ]:
# Test with df returns.
if __name__ == "__main__":
  cal = read_metashape_camera_cal_file( metashape_cal_filename, units='m', rv='df' )
  display(cal)

,filename,pixel_size,units,projection,width,height,f,cx,cy,k1,k2,k3,p1,p2,date
0,../test_data/we-1500-A1-agisoft-camcal.xml,0.000005,m,frame,0.026928,0.017952,0.028752,0.000065,-0.000048,-0.094197,0.115036,-0.032238,-0.000257,-0.000354,2024-04-04 04:49:48


#| hide
### def read_openCV_camera_cal_file( filename, debug=False )

In [ ]:
#|export
class read_openCV_camera_cal_file:
  """
  Read an OpenCV Camera xml Calibration file.
  """
  def __init__(self, filename ):
    tree = ET.parse( filename )
    root = tree.getroot()
    tree.getroot()

    #self.filename = filename.split("/")[-1]
    filepath = Path( filename )
    self.filename  = filepath  #filepath.name
    self.datetime = root.findall('calibration_Time')[0].text
    self.image_Width = root.findall('image_Width')[0].text
    self.image_Height = root.findall('image_Height')[0].text

    self.Camera_Matrix_rows = int(root.findall('Camera_Matrix/rows')[0].text)
    self.Camera_Matrix_cols = int(root.findall('Camera_Matrix/cols')[0].text)
    self.Camera_Matrix_data = root.findall('Camera_Matrix/data')[0].text
    lst = root.findall('Camera_Matrix/data')[0].text.strip()
    x = re.split('[ \n]+', lst )
    self.Camera_Matrix_data = np.array(x, dtype=float)
    self.Camera_Matrix_data.reshape( self.Camera_Matrix_rows,self.Camera_Matrix_cols)

    self.Distortion_Coefficients_rows = int(root.findall('Distortion_Coefficients/rows')[0].text)
    self.Distortion_Coefficients_cols = int(root.findall('Distortion_Coefficients/cols')[0].text)
    lst = root.findall('Distortion_Coefficients/data')[0].text.strip()
    x = re.split('[ \n]+', lst )
    self.Distortion_Coefficients_data = np.array(x, dtype=float)

    # Add as a MetaShape class.
    self.metashape = MetaShape_Cal_Data()
    self.metashape.units        = 'pixels'
    self.metashape.pixel_size   = 5.5e-6
    self.metashape.projection   = 'frame'
    self.metashape.filename     = filepath
    self.metashape.datetime     = self.datetime
    self.metashape.width        = self.image_Width
    self.metashape.height       = self.image_Height
    self.metashape.f            = self.Camera_Matrix_data[0]
    self.metashape.cx           = self.Camera_Matrix_data[2]
    self.metashape.cy           = self.Camera_Matrix_data[5]
    self.metashape.k1           = self.Distortion_Coefficients_data[0]
    self.metashape.k2           = self.Distortion_Coefficients_data[1]
    self.metashape.k3           = self.Distortion_Coefficients_data[4]
    self.metashape.p1           = self.Distortion_Coefficients_data[3]
    self.metashape.p2           = self.Distortion_Coefficients_data[2]

#| hide
#### Test read_openCV_camera_cal_file()

In [ ]:
if __name__ == "__main__":
  cal = read_openCV_camera_cal_file(opencv_filename)

  # Dump the data.
  print("\n**** openCV elements. ****")
  for k, v in cal.__dict__.items():
    print(f'{k:32s} = {v}')

  # Dump the metashape data.
  print("\n**** MetaShape elements. ****")
  for k, v in cal.metashape.__dict__.items():
    print(f'{k:32s} = {v}')

  print("\n**** Example reading elements directly ****")
  print(f'{cal.metashape.width = }')


**** openCV elements. ****
filename                         = ../test_data/we-1500-A1-opencv-camcal.xml
datetime                         = "Thu Apr  4 15:40:46 2024"
image_Width                      = 4896
image_Height                     = 3264
Camera_Matrix_rows               = 3
Camera_Matrix_cols               = 3
Camera_Matrix_data               = [5.22763323e+03 0.00000000e+00 2.45923870e+03 0.00000000e+00
 5.22763323e+03 1.62280113e+03 0.00000000e+00 0.00000000e+00
 1.00000000e+00]
Distortion_Coefficients_rows     = 5
Distortion_Coefficients_cols     = 1
Distortion_Coefficients_data     = [-0.09419663  0.11503642 -0.00035361 -0.00025662 -0.03223831]
metashape                        = <__main__.MetaShape_Cal_Data object>

**** MetaShape elements. ****
filename                         = ../test_data/we-1500-A1-opencv-camcal.xml
datetime                         = "Thu Apr  4 15:40:46 2024"
projection                       = frame
units                            = pixels
pixel_siz

#| hide
### def metashape_cal_to_tsai()

In [ ]:
#|export
def metashape_cal_to_tsai( df,               # MetaShape cal dataframe.
                          save=True,         # True to save tasi to a file.
                           tsai_file='',     # An optional filename to write the tsai data too.
                           path=".",         # Path to save the file to.
                           return_str=False, # True if you want the tsai string returned.
                           debug=False       # True debug tis module.
                           ) -> str:         # Returns the entire tsai cal string.
  """
  Convert a `MetaShape` camera calibration Pandas Dataframe to a tsai file.
  """

  # Compute the tsai values from the Metashape cal dataframe.
  fu  = fv = df.f[0]
  cu  = df.width[0]  / 2 + 0.5 - 1.0 + df.cx[0]
  cv  = df.height[0] / 2 + 0.5 - 1.0 + df.cy[0]
  k1  = df.k1[0];   k2 = df.k2[0];   p1 = df.p1[0];   p2 = df.p2[0]

  # Generate tsai string.
  tsai_str = \
       "VERSION_4\nPINHOLE\n"\
      f"{fu = :15.12f}\n{fv = :15.12f}\n{cu = :15.12f}\n{cv = :15.12f}\n"\
      "u_direction = 1  0  0\n"\
      "v_direction = 0  1  0\n"\
      "w_direction = 0  0  1\n"\
      "C = 0 0 0\nR = 1 0 0 0 1 0 0 0 1\n"\
      "pitch = 1.0\nTSAI\n"\
      f"{k1 = :15.12f}\n{k2 = :15.12f}\n{p1 = :15.12f}\n{p2 = :15.12f}\n"

  # Save it to a file.  Generate the file name from the agisoft filename.
  if save:
    if tsai_file:
      tsai_fn = Path(path+"/"+tsai_file)
      if debug:
        print(f'Debug: {tsai_fn=}')
    else:
      # tsai_fn = df.filename[0].split(".")[0]+".tsai"
      tsai_fn = Path(path+"/"+df.filename[0].name).with_suffix('.tsai')
      if debug:
        print(f'Debug: {tsai_fn=}')
      # tsai_fn.with_suffix('.tsai')
    with open( tsai_fn, 'w') as tsai_file:
      tsai_file.write( tsai_str )

  if debug:
    print(f'debug: {tsai_str:s}')

  # Return the tsai string if return_str=True
  if return_str:
    return tsai_str

#| hide
#### Test def metashape_cal_to_tsai()

test metashape_cal_to_tsai( cal )

In [ ]:
# test metashape_cal_to_tsai( cal )
if __name__ == "__main__":
  cal = read_metashape_camera_cal_file( metashape_cal_filename, units='m', rv='df', debug=True )
  display(cal)
  print(cal.filename)
  metashape_cal_to_tsai( cal )

debug: root.tag:calibration
debug: projection = frame
debug: width      = 4896
debug: height     = 3264
debug: f          = 5227.6332278443551
debug: cx         = 11.738704269580627
debug: cy         = -8.6988693925254026
debug: k1         = -0.094196634563145282
debug: k2         = 0.11503642426152168
debug: k3         = -0.032238313340550974
debug: p1         = -0.00025662254090614374
debug: p2         = -0.00035361346010062737
debug: date       = 2024-04-04T04:49:48Z


,filename,pixel_size,units,projection,width,height,f,cx,cy,k1,k2,k3,p1,p2,date
0,../test_data/we-1500-A1-agisoft-camcal.xml,0.000005,m,frame,0.026928,0.017952,0.028752,0.000065,-0.000048,-0.094197,0.115036,-0.032238,-0.000257,-0.000354,2024-04-04 04:49:48


0    ../test_data/we-1500-A1-agisoft-camcal.xml
Name: filename, dtype: object


Test generating a user specified file name.

In [ ]:
# Test with user set file name.
if __name__ == "__main__":
  metashape_cal_to_tsai( cal, tsai_file="junk.tsai" )

In [ ]:
# Test with automatically generated filename
if __name__ == "__main__":
  metashape_cal_to_tsai( cal, debug=True )

Debug: tsai_fn=PosixPath('we-1500-A1-agisoft-camcal.tsai')
debug: VERSION_4
PINHOLE
fu =  0.028751982753
fv =  0.028751982753
cu = -0.486471437127
cv = -0.491071843782
u_direction = 1  0  0
v_direction = 0  1  0
w_direction = 0  0  1
C = 0 0 0
R = 1 0 0 0 1 0 0 0 1
pitch = 1.0
TSAI
k1 = -0.094196634563
k2 =  0.115036424262
p1 = -0.000256622541
p2 = -0.000353613460



Test with user set path. Write the test file to a user specified path `path='/tmp'`

In [ ]:
# Test with user set path. Write the test file to /tmp
if __name__ == "__main__":
  metashape_cal_to_tsai( cal, path="/tmp" )

#| hide
## Experimentation Code

In [ ]:
#| hide
if __name__ == "__main__" and False:
  fn = Path("/dddd"+'/content/drive/MyDrive/Colab Notebooks/WWpylib/test_data/we-1500-A1-opencv-camcal.xml')
  display(fn.parts)

In [ ]:
#| hide
if __name__ == "__main__" and False:
  fn = fn.with_suffix('.tsai')
  display(fn)

In [ ]:
#| hide
# Testing cell.
if __name__ == "__main__" and False:
  filename = Path('/content/drive/MyDrive/Colab Notebooks/WWpylib/test_data/we-1500-A1-agisoft-camcal.xml')
  wp = PureWindowsPath(r"C:\Users\mstuding\OneDrive - NASA\GitHub_repositories\ATM-SfM-Production")
  display(wp.parts)

## References

1. **[Medium Article: pathlib](https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f)** path(), PurePosixPath() and PureWindowsPath()
2. **[nbdev directives.](https://nbdev.fast.ai/explanations/directives.html)** A cheat sheet of directives available in nbdev.